# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
from workspace_utils import active_session

import time
import copy

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import matplotlib.pyplot as plt
%matplotlib inline

import PIL
from PIL import Image
import numpy as np

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomVerticalFlip(),
                                       transforms.RandomRotation(45),
                                       transforms.RandomResizedCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                            std=[0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                           std=[0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(256),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train = datasets.ImageFolder(train_dir, transform=train_transforms)
test = datasets.ImageFolder(test_dir, transform=test_transforms)
validation = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test, batch_size=64)
validloader = torch.utils.data.DataLoader(validation, batch_size=64)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [ ]:
class Network(nn.Module):
    def __init__(self, input_size, output_size, hidden_sizes, drop_p=0.3):
        super().__init__()
                
        # Inputs to hidden layer linear transformation
        self.hidden_sizes = nn.ModuleList([nn.Linear(input_size, hidden_sizes[0])])
        
        # Add a variable number of more hidden layers
        layer_sizes = zip(hidden_sizes[:-1], hidden_sizes[1:])
        self.hidden_sizes.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        # Output layer
        self.output = nn.Linear(hidden_sizes[-1], output_size)
        
        self.dropout = nn.Dropout(p=drop_p)
        
    def forward(self, x):
        ''' Forward pass through the network, returns the output logits '''
        
        for each in self.hidden_sizes:
            
            x = F.relu(each(x))
            x = self.dropout(x)
        
        x = self.output(x)
        
        return F.log_softmax(x, dim=1)

In [ ]:
def train_model(model, criterion, optimizer, num_epochs, model_path=None):
    
    if model_path is None:
        start = 0
        iterations = num_epochs
        train_losses, valid_losses = [], []

    else:
        model, optimizer, iterations, criterion, train_losses, valid_losses = load_checkpoint(model_path)
        model = model.to(device)
        start = iterations
        iterations = num_epochs + start
        print('num_epochs: {}'.format(num_epochs))
        print('iterations: {}'.format(iterations))
        
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    since = time.time()
    steps = 0
    model.to(device)
    
    for epoch in range(start, iterations):
        print('Epoch {}/{}'.format(epoch+1, iterations))
        print('-' * 10)
        print("Train losses: {}".format(train_losses))
        print("Valid losses: {}".format(valid_losses))
        running_loss = 0
        model.train()
        
        for images, labels in trainloader:
            since_train_step = time.time()
            steps += 1
            
            # Move input and label tensors to the GPU
            images, labels = images.to(device), labels.to(device)

            model.train()  
            optimizer.zero_grad()

            log_ps = model(images)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            print("Time per train step {}/{}: {}".format(steps, len(trainloader), time.time() - since_train_step))

        else: 
            # Model in inference mode, dropout is off
            model.eval()

            # Turn off gradients for validation, will speed up inference
            with torch.no_grad():
                valid_loss, accuracy = validation(model, validloader, criterion)

            train_losses.append(round(running_loss/len(trainloader), 3))
            valid_losses.append(round(valid_loss/len(validloader), 3))
            
            if accuracy > best_acc:
                best_acc = accuracy
                best_model_wts = copy.deepcopy(model.state_dict())

            print("Epoch: {}/{}.. ".format(epoch+1, iterations),
                  "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
                  "Test Loss: {:.3f}.. ".format(valid_loss/len(validloader)),
                  "Test Accuracy: {:.3f}..".format(accuracy/len(validloader)))

            running_loss = 0
            steps = 0

            # Make sure dropout and grads are on for training
            model.train()
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return model, optimizer, iterations, train_losses, valid_losses

In [ ]:
def validation(model, validloader, criterion):
    
    accuracy = 0
    valid_loss = 0
    since_test_step = time.time()
    
    for images, labels in validloader:
        
        images, labels = images.to(device), labels.to(device)
        output = model.forward(images)
        batch_loss = criterion(output, labels)
        valid_loss += batch_loss.item()
        
        ## Calculating the accuracy 
        # Model's output is log-softmax, take exponential to get the probabilities
        ps = torch.exp(output)
        # Class with highest probability is our predicted class
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))

        print("Time per test step: {}".format(time.time() - since_test_step))

    return valid_loss, accuracy

In [ ]:
# TODO: Build and train your network
with active_session():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = models.vgg16(pretrained=True)
    model.class_to_idx = train.class_to_idx

    #turn off gradients for the model
    for param in model.parameters():
        param.requires_grad = False

    input_size = 25088
    output_size = 102
    hidden_sizes = [256, 128]

    #define our new classifier
    model.classifier = Network(input_size, output_size, hidden_sizes)

    criterion = nn.NLLLoss()

    #train classifier parameters
    learning_rate = 0.003
    optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)
    
    model, optimizer, iterations, train_losses, valid_losses = train_model(model, criterion, optimizer, num_epochs=12, model_path='checkpoint_vgg16_3epochs.pth')

In [ ]:
# plot the training loss and validation loss to check for overfitting
plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.legend(frameon=False);

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
checkpoint = {'input_size': input_size,
              'output_size': output_size,
              'hidden_sizes': hidden_sizes,
              'state_dict': model.state_dict(),
              'optimizer': optimizer.state_dict(),
              'class_to_idx': model.class_to_idx,
              'iterations': iterations,
              'learning_rate': learning_rate,
              'criterion': criterion,
              'train_losses': train_losses,
              'valid_losses': valid_losses}

torch.save(checkpoint, 'checkpoint_vgg16_3epochs.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    
    #commented code is for the model that was saved on 'cpu'. then we need to move optimizer to cuda (below)
    checkpoint = torch.load(filepath, map_location='cpu')
    #checkpoint = torch.load(filepath)
    model = models.vgg16(pretrained=True)
    model.classifier = Network(checkpoint['input_size'],
                               checkpoint['output_size'],
                               checkpoint['hidden_sizes'])
    
    model.load_state_dict(checkpoint['state_dict'])
    
    optimizer = optim.Adam(model.classifier.parameters(), lr = checkpoint['learning_rate'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    #for state in optimizer.state.values():
    #    for k, v in state.items():
    #        if torch.is_tensor(v):
    #            state[k] = v.cuda()
    model.class_to_idx = checkpoint['class_to_idx']
    iterations = checkpoint['iterations']
    criterion = checkpoint['criterion']
    train_losses = checkpoint['train_losses']
    valid_losses = checkpoint['valid_losses']
    
    return model, optimizer, iterations, criterion, train_losses, valid_losses

In [ ]:
model, optimizer, iterations, criterion, train_losses, valid_losses = load_checkpoint('checkpoint_vgg16_3epochs.pth')
print(iterations)
print(train_losses)
print(valid_losses)
print(model)
print(optimizer.state_dict)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    image = Image.open(image)
    image.thumbnail([256, 256])
    
    torch_cropped = transforms.CenterCrop((224,224))
    center_cropped = torch_cropped(image)
 
    np_image = np.array(center_cropped)/256
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    norm_image = (np_image - mean)/std
    trans_image = norm_image.transpose((2, 0, 1))
    
    return trans_image
    
    # TODO: Process a PIL image for use in a PyTorch model

In [ ]:
torched = process_image('flowers/test/6/image_07182.jpg')

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')
    
    return ax

In [ ]:
imshow(torched);

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    img_numpy = process_image(image_path)
    img = torch.from_numpy(img_numpy)
    
    model.eval()
    # Turn off gradients to speed up this part
    with torch.no_grad():
        model.double()
        logits = model.forward(img.unsqueeze_(0))
    
    # Output of the network are logits, need to take exponential for probabilities
    ps = torch.exp(logits)

    probs, class_idx = ps.topk(5, dim=1)
    probs = probs.data.numpy().squeeze()
    class_idx = class_idx.data.numpy().squeeze()
    
    i = 0
    classes = []
    for i in range(class_idx.size):
        for key, value in model.class_to_idx.items():
            if value == class_idx[i]:
                classes.append(key)
        i += 1

    pred_class_names = []
    for x in classes:
        y = cat_to_name[x]
        pred_class_names.append(y)
        
    return probs, pred_class_names
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
def view_classify(img_path, model):
    ''' Function for viewing an image and it's predicted classes.
    '''
    probs, pred_class_names = predict(img_path, model)

    img = Image.open(img_path)
    img.thumbnail([256, 256])

    torch_cropped = transforms.CenterCrop((224,224))
    center_cropped = torch_cropped(img)
    
    cat_from_path =  str(img_path.split('/')[-2])
    
    fig, (ax1, ax2) = plt.subplots(figsize=(5,7), nrows=2)
    ax1.imshow(center_cropped)
    ax1.axis('off')
    ax2.set_aspect(0.18)
    ax1.set_title(cat_to_name[cat_from_path])
    ax2.set_yticks(np.arange(5))
    ax2.set_yticklabels(pred_class_names)
    ax2.barh(np.arange(5), probs)
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1)
    fig.tight_layout();

In [ ]:
view_classify('flowers/test/6/image_07182.jpg', model)